# Logic

This Jupyter notebook acts as supporting material for topics covered in __Chapter 6 Logical Agents__ of the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. We make use of the implementations in the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

Let's first import everything from the `logic` module.

In [1]:
from utils import *
from logic import *

## CONTENTS
- Logical sentences
    - Expr
    - PropKB
    - Knowledge-based agents
    - Inference in propositional knowledge base
        - Truth table enumeration
        - Proof by resolution
        - Forward and backward chaining
        - DPLL

## Logical Sentences

The `Expr` class is designed to represent any kind of mathematical expression. The simplest type of `Expr` is a symbol, which can be defined with the function `Symbol`:

In [2]:
Symbol('x')

x

Or we can define multiple symbols at the same time with the function `symbols`:

In [3]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

We can combine `Expr`s with the regular Python infix and prefix operators. Here's how we would form the logical sentence "P and not Q":

In [4]:
P & ~Q

(P & ~Q)

This works because the `Expr` class overloads the `&` operator with this definition:

```python
def __and__(self, other): return Expr('&',  self, other)```
     
and does similar overloads for the other operators. An `Expr` has two fields: `op` for the operator, which is always a string, and `args` for the arguments, which is a tuple of 0 or more expressions. By "expression," I mean either an instance of `Expr`, or a number. Let's take a look at the fields for some `Expr` examples:

In [5]:
sentence = P & ~Q

sentence.op

'&'

In [6]:
sentence.args

(P, ~Q)

In [7]:
P.op

'P'

In [8]:
P.args

()

In [9]:
Pxy = P(x, y)

Pxy.op

'P'

In [10]:
Pxy.args

(x, y)

It is important to note that the `Expr` class does not define the *logic* of Propositional Logic sentences; it just gives you a way to *represent* expressions. Think of an `Expr` as an [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree).  Each of the `args` in an `Expr` can be either a symbol, a number, or a nested `Expr`. We can nest these trees to any depth. Here is a deply nested `Expr`:

In [11]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operators for Constructing Logical Sentences

Here is a table of the operators that can be used to form sentences. Note that we have a problem: we want to use Python operators to make sentences, so that our programs (and our interactive sessions like the one here) will show simple code. But Python does not allow implication arrows as operators, so for now we have to use a more verbose notation that Python does allow: `|'==>'|` instead of just `==>`. Alternately, you can always use the more verbose `Expr` constructor forms:

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

Here's an example of defining a sentence with an implication arrow:

In [12]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

## `expr`: a Shortcut for Constructing Sentences

If the `|'==>'|` notation looks ugly to you, you can use the function `expr` instead:

In [13]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

`expr` takes a string as input, and parses it into an `Expr`. The string can contain arrow operators: `==>`, `<==`, or `<=>`, which are handled as if they were regular Python infix operators. And `expr` automatically defines any symbols, so you don't need to pre-define them:

In [14]:
expr('sqrt(b ** 2 - 4 * a * c)')

sqrt(((b ** 2) - ((4 * a) * c)))

For now that's all you need to know about `expr`. If you are interested, we explain the messy details of how `expr` is implemented and how `|'==>'|` is handled in the appendix.

## Propositional Knowledge Bases: `PropKB`

The class `PropKB` can be used to represent a knowledge base of propositional logic sentences.

We see that the class `KB` has four methods, apart from `__init__`. A point to note here: the `ask` method simply calls the `ask_generator` method. Thus, this one has already been implemented, and what you'll have to actually implement when you create your own knowledge base class (though you'll probably never need to, considering the ones we've created for you) will be the `ask_generator` function and not the `ask` function itself.

The class `PropKB` now.
* `__init__(self, sentence=None)` : The constructor `__init__` creates a single field `clauses` which will be a list of all the sentences of the knowledge base. Note that each one of these sentences will be a 'clause' i.e. a sentence which is made up of only literals and `or`s.
* `tell(self, sentence)` : When you want to add a sentence to the KB, you use the `tell` method. This method takes a sentence, converts it to its CNF, extracts all the clauses, and adds all these clauses to the `clauses` field. So, you need not worry about `tell`ing only clauses to the knowledge base. You can `tell` the knowledge base a sentence in any form that you wish; converting it to CNF and adding the resulting clauses will be handled by the `tell` method.
* `ask_generator(self, query)` : The `ask_generator` function is used by the `ask` function. It calls the `tt_entails` function, which in turn returns `True` if the knowledge base entails query and `False` otherwise. The `ask_generator` itself returns an empty dict `{}` if the knowledge base entails query and `None` otherwise. This might seem a little bit weird to you. After all, it makes more sense just to return a `True` or a `False` instead of the `{}` or `None` But this is done to maintain consistency with the way things are in First-Order Logic, where an `ask_generator` function is supposed to return all the substitutions that make the query true. Hence the dict, to return all these substitutions. I will be mostly be using the `ask` function which returns a `{}` or a `False`, but if you don't like this, you can always use the `ask_if_true` function which returns a `True` or a `False`.
* `retract(self, sentence)` : This function removes all the clauses of the sentence given, from the knowledge base. Like the `tell` function, you don't have to pass clauses to remove them from the knowledge base; any sentence will do fine. The function will take care of converting that sentence to clauses and then remove those.

## Wumpus World KB
Let us create a `PropKB` for the wumpus world with the sentences mentioned in `section 7.4.3`.

In [15]:
wumpus_kb = PropKB()

We define the symbols we use in our clauses.<br/>
$P_{x, y}$ is true if there is a pit in `[x, y]`.<br/>
$B_{x, y}$ is true if the agent senses breeze in `[x, y]`.<br/>

In [16]:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

Now we tell sentences based on `section 7.4.3`.<br/>
There is no pit in `[1,1]`.

In [17]:
wumpus_kb.tell(~P11)

A square is breezy if and only if there is a pit in a neighboring square. This has to be stated for each square but for now, we include just the relevant squares.

In [18]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Now we include the breeze percepts for the first two squares leading up to the situation in `Figure 7.3(b)`

In [19]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

We can check the clauses stored in a `KB` by accessing its `clauses` variable

In [20]:
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

We see that the equivalence $B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was automatically converted to two implications which were inturn converted to CNF which is stored in the `KB`.<br/>
$B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was split into $B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ and $B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$.<br/>
$B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ was converted to $P_{1, 2} \lor P_{2, 1} \lor \neg B_{1, 1}$.<br/>
$B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$ was converted to $\neg (P_{1, 2} \lor P_{2, 1}) \lor B_{1, 1}$ which becomes $(\neg P_{1, 2} \lor B_{1, 1}) \land (\neg P_{2, 1} \lor B_{1, 1})$ after applying De Morgan's laws and distributing the disjunction.<br/>
$B_{2, 1} \iff (P_{1, 1} \lor P_{2, 2} \lor P_{3, 2})$ is converted in similar manner.

### Proof by Resolution
Recall that our goal is to check whether $\text{KB} \vDash \alpha$ i.e. is $\text{KB} \implies \alpha$ true in every model. Suppose we wanted to check if $P \implies Q$ is valid. We check the satisfiability of $\neg (P \implies Q)$, which can be rewritten as $P \land \neg Q$. If $P \land \neg Q$ is unsatisfiable, then $P \implies Q$ must be true in all models. This gives us the result "$\text{KB} \vDash \alpha$ <em>if and only if</em> $\text{KB} \land \neg \alpha$ is unsatisfiable".<br/>
This technique corresponds to <em>proof by <strong>contradiction</strong></em>, a standard mathematical proof technique. We assume $\alpha$ to be false and show that this leads to a contradiction with known axioms in $\text{KB}$. We obtain a contradiction by making valid inferences using inference rules. In this proof we use a single inference rule, <strong>resolution</strong> which states $(l_1 \lor \dots \lor l_k) \land (m_1 \lor \dots \lor m_n) \land (l_i \iff \neg m_j) \implies l_1 \lor \dots \lor l_{i - 1} \lor l_{i + 1} \lor \dots \lor l_k \lor m_1 \lor \dots \lor m_{j - 1} \lor m_{j + 1} \lor \dots \lor m_n$. Applying the resolution yields us a clause which we add to the KB. We keep doing this until:

* There are no new clauses that can be added, in which case $\text{KB} \nvDash \alpha$.
* Two clauses resolve to yield the <em>empty clause</em>, in which case $\text{KB} \vDash \alpha$.

The <em>empty clause</em> is equivalent to <em>False</em> because it arises only from resolving two complementary
unit clauses such as $P$ and $\neg P$ which is a contradiction as both $P$ and $\neg P$ can't be <em>True</em> at the same time.

There is  one catch however, the algorithm that implements proof by resolution cannot handle complex sentences. 
Implications and bi-implications have to be simplified into simpler clauses. 
We already know that *every sentence of a propositional logic is logically equivalent to a conjunction of clauses*.
We will use this fact to our advantage and simplify the input sentence into the **conjunctive normal form** (CNF) which is a conjunction of disjunctions of literals.
For eg:
<br>
$$(A\lor B)\land (\neg B\lor C\lor\neg D)\land (D\lor\neg E)$$
This is equivalent to the POS (Product of sums) form in digital electronics.
<br>
Here's an outline of how the conversion is done:
1. Convert bi-implications to implications
<br>
$\alpha\iff\beta$ can be written as $(\alpha\implies\beta)\land(\beta\implies\alpha)$
<br>
This also applies to compound sentences
<br>
$\alpha\iff(\beta\lor\gamma)$ can be written as $(\alpha\implies(\beta\lor\gamma))\land((\beta\lor\gamma)\implies\alpha)$
<br>
2. Convert implications to their logical equivalents
<br>
$\alpha\implies\beta$ can be written as $\neg\alpha\lor\beta$
<br>
3. Move negation inwards
<br>
CNF requires atomic literals. Hence, negation cannot appear on a compound statement.
De Morgan's laws will be helpful here.
<br>
$\neg(\alpha\land\beta)\equiv(\neg\alpha\lor\neg\beta)$
<br>
$\neg(\alpha\lor\beta)\equiv(\neg\alpha\land\neg\beta)$
<br>
4. Distribute disjunction over conjunction
<br>
Disjunction and conjunction are distributive over each other.
Now that we only have conjunctions, disjunctions and negations in our expression, 
we will distribute disjunctions over conjunctions wherever possible as this will give us a sentence which is a conjunction of simpler clauses, 
which is what we wanted in the first place.
<br>
We need a term of the form
<br>
$(\alpha_{1}\lor\alpha_{2}\lor\alpha_{3}...)\land(\beta_{1}\lor\beta_{2}\lor\beta_{3}...)\land(\gamma_{1}\lor\gamma_{2}\lor\gamma_{3}...)\land...$
<br>
<br>
The `to_cnf` function executes this conversion using helper subroutines.

`to_cnf` calls three subroutines.
<br>
`eliminate_implications` converts bi-implications and implications to their logical equivalents.
<br>
`move_not_inwards` removes negations from compound statements and moves them inwards using De Morgan's laws.
<br>
`distribute_and_over_or` distributes disjunctions over conjunctions.
<br>
Run the cell below for implementation details.

Let's convert some sentences to see how it works


In [21]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [22]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [23]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [24]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

Coming back to our resolution problem, we can see how the `to_cnf` function is utilized here

In [25]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)

In [26]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

(False, False)

### Forward and backward chaining
Previously, we said we will look at two algorithms to check if a sentence is entailed by the `KB`. Here's a third one. 
The difference here is that our goal now is to determine if a knowledge base of definite clauses entails a single proposition symbol *q* - the query.
There is a catch however - the knowledge base can only contain **Horn clauses**.
<br>
#### Horn Clauses
Horn clauses can be defined as a *disjunction* of *literals* with **at most** one positive literal. 
<br>
A Horn clause with exactly one positive literal is called a *definite clause*.
<br>
A Horn clause might look like 
<br>
$\neg a\lor\neg b\lor\neg c\lor\neg d... \lor z$
<br>
This, coincidentally, is also a definite clause.
<br>
Using De Morgan's laws, the example above can be simplified to 
<br>
$a\land b\land c\land d ... \implies z$
<br>
This seems like a logical representation of how humans process known data and facts. 
Assuming percepts `a`, `b`, `c`, `d` ... to be true simultaneously, we can infer `z` to also be true at that point in time. 
There are some interesting aspects of Horn clauses that make algorithmic inference or *resolution* easier.
- Definite clauses can be written as implications:
<br>
The most important simplification a definite clause provides is that it can be written as an implication.
The premise (or the knowledge that leads to the implication) is a conjunction of positive literals.
The conclusion (the implied statement) is also a positive literal.
The sentence thus becomes easier to understand.
The premise and the conclusion are conventionally called the *body* and the *head* respectively.
A single positive literal is called a *fact*.
- Forward chaining and backward chaining can be used for inference from Horn clauses:
<br>
Forward chaining is semantically identical to `AND-OR-Graph-Search` from the chapter on search algorithms.
Implementational details will be explained shortly.
- Deciding entailment with Horn clauses is linear in size of the knowledge base:
<br>
Surprisingly, the forward and backward chaining algorithms traverse each element of the knowledge base at most once, greatly simplifying the problem.
<br>
<br>
The function `pl_fc_entails` implements forward chaining to see if a knowledge base `KB` entails a symbol `q`.
<br>
Before we proceed further, note that `pl_fc_entails` doesn't use an ordinary `KB` instance. 
The knowledge base here is an instance of the `PropDefiniteKB` class, derived from the `PropKB` class, 
but modified to store definite clauses.
<br>
The main point of difference arises in the inclusion of a helper method to `PropDefiniteKB` that returns a list of clauses in KB that have a given symbol `p` in their premise.

Let's now have a look at the `pl_fc_entails` algorithm.

The function accepts a knowledge base `KB` (an instance of `PropDefiniteKB`) and a query `q` as inputs.
<br>
<br>
`count` initially stores the number of symbols in the premise of each sentence in the knowledge base.
<br>
The `conjuncts` helper function separates a given sentence at conjunctions.
<br>
`inferred` is initialized as a *boolean* defaultdict. 
This will be used later to check if we have inferred all premises of each clause of the agenda.
<br>
`agenda` initially stores a list of clauses that the knowledge base knows to be true.
The `is_prop_symbol` helper function checks if the given symbol is a valid propositional logic symbol.
<br>
<br>
We now iterate through `agenda`, popping a symbol `p` on each iteration.
If the query `q` is the same as `p`, we know that entailment holds.
<br>
The agenda is processed, reducing `count` by one for each implication with a premise `p`.
A conclusion is added to the agenda when `count` reaches zero. This means we know all the premises of that particular implication to be true.
<br>
`clauses_with_premise` is a helpful method of the `PropKB` class.
It returns a list of clauses in the knowledge base that have `p` in their premise.
<br>
<br>
Now that we have an idea of how this function works, let's see a few examples of its usage, but we first need to define our knowledge base. We assume we know the following clauses to be true.

In [27]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']

We will now `tell` this information to our knowledge base.

In [28]:
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

We can now check if our knowledge base entails the following queries.

In [29]:
pl_fc_entails(definite_clauses_KB, expr('G'))

True

In [30]:
pl_fc_entails(definite_clauses_KB, expr('H'))

True

In [31]:
pl_fc_entails(definite_clauses_KB, expr('I'))

False

In [32]:
pl_fc_entails(definite_clauses_KB, expr('J'))

False

### Effective Propositional Model Checking

The previous segments elucidate the algorithmic procedure for model checking. 
In this segment, we look at ways of making them computationally efficient.
<br>
The problem we are trying to solve is conventionally called the _propositional satisfiability problem_, abbreviated as the _SAT_ problem.
In layman terms, if there exists a model that satisfies a given Boolean formula, the formula is called satisfiable.
<br>
The SAT problem was the first problem to be proven _NP-complete_.
The main characteristics of an NP-complete problem are:
- Given a solution to such a problem, it is easy to verify if the solution solves the problem.
- The time required to actually solve the problem using any known algorithm increases exponentially with respect to the size of the problem.
<br>
<br>
Due to these properties, heuristic and approximational methods are often applied to find solutions to these problems.
<br>
It is extremely important to be able to solve large scale SAT problems efficiently because 
many combinatorial problems in computer science can be conveniently reduced to checking the satisfiability of a propositional sentence under some constraints.
<br>
We will introduce two new algorithms that perform propositional model checking in a computationally effective way.
<br>


### 1. DPLL (Davis-Putnam-Logeman-Loveland) algorithm
This algorithm is very similar to Backtracking-Search.
It recursively enumerates possible models in a depth-first fashion with the following improvements over algorithms like `tt_entails`:
1. Early termination:
<br>
In certain cases, the algorithm can detect the truth value of a statement using just a partially completed model.
For example, $(P\lor Q)\land(P\lor R)$ is true if P is true, regardless of other variables.
This reduces the search space significantly.
2. Pure symbol heuristic:
<br>
A symbol that has the same sign (positive or negative) in all clauses is called a _pure symbol_.
It isn't difficult to see that any satisfiable model will have the pure symbols assigned such that its parent clause becomes _true_.
For example, $(P\lor\neg Q)\land(\neg Q\lor\neg R)\land(R\lor P)$ has P and Q as pure symbols
and for the sentence to be true, P _has_ to be true and Q _has_ to be false.
The pure symbol heuristic thus simplifies the problem a bit.
3. Unit clause heuristic:
<br>
In the context of DPLL, clauses with just one literal and clauses with all but one _false_ literals are called unit clauses.
If a clause is a unit clause, it can only be satisfied by assigning the necessary value to make the last literal true.
We have no other choice.
<br>
Assigning one unit clause can create another unit clause.
For example, when P is false, $(P\lor Q)$ becomes a unit clause, causing _true_ to be assigned to Q.
A series of forced assignments derived from previous unit clauses is called _unit propagation_.
In this way, this heuristic simplifies the problem further.
<br>
The algorithm often employs other tricks to scale up to large problems.
However, these tricks are currently out of the scope of this notebook. Refer to section 7.6 of the book for more details.
<br>
<br>
Let's have a look at the algorithm.

The algorithm uses the ideas described above to check satisfiability of a sentence in propositional logic.
It recursively calls itself, simplifying the problem at each step. It also uses helper functions `find_pure_symbol` and `find_unit_clause` to carry out steps 2 and 3 above.
<br>
The `dpll_satisfiable` helper function converts the input clauses to _conjunctive normal form_ and calls the `dpll` function with the correct parameters.

Let's see a few examples of usage.

In [33]:
A, B, C, D = expr('A, B, C, D')

In [34]:
dpll_satisfiable(A & B & ~C & D)

{C: False, B: True, D: True, A: True}

This is a simple case to highlight that the algorithm actually works.

In [35]:
dpll_satisfiable((A & B) | (C & ~A) | (B & ~D))

{C: True, B: True, D: False}

If a particular symbol isn't present in the solution, 
it means that the solution is independent of the value of that symbol.
In this case, the solution is independent of A.

In [36]:
dpll_satisfiable(A |'<=>'| B)

{B: True, A: True}

In [37]:
dpll_satisfiable((A |'<=>'| B) |'==>'| (C & ~A))

{C: True, B: True, A: False}

In [38]:
dpll_satisfiable((A | (B & C)) |'<=>'| ((A | B) & (A | C)))

{C: True, B: True}

In [39]:
dpll_satisfiable((~P11&(B11|'<=>'|(P12|P21))&(B21|'<=>'|(P11|P22|P31))&~B11&B21))

{P11: False, B11: False, P21: False, P12: False, B21: True, P22: True}

# Authors

This notebook by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig).

